<a href="https://colab.research.google.com/github/raspberryscorn/2023/blob/main/Style_Transfer_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Style Transfer



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## imports

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import tensorflow_hub as hub
import urllib.request

## download Image

In [ ]:
def download_img(url, save_as):
    urllib.request.urlretrieve(url, f"{save_as}.jpg")

In [ ]:
content_url = 'https://github.com/onebottlekick/JNU_dl/releases/download/dataset/content.jpg'
style_url = 'https://github.com/onebottlekick/JNU_dl/releases/download/dataset/style.jpg'

download_img(content_url, 'content')
download_img(style_url, 'style')

## dataset

In [ ]:
class DataReader():
    def __init__(self, content, style):
        self.content = self.load_img(content)
        self.style = self.load_img(style)

    def load_img(self, path_to_img):
      max_dim = 512
      img = tf.io.read_file(path_to_img)
      img = tf.image.decode_image(img, channels=3)
      img = tf.image.convert_image_dtype(img, tf.float32)

      shape = tf.cast(tf.shape(img)[:-1], tf.float32)
      long_dim = max(shape)
      scale = max_dim / long_dim

      new_shape = tf.cast(shape * scale, tf.int32)

      img = tf.image.resize(img, new_shape)
      img = img[tf.newaxis, :]
      return tf.constant(img)


def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return Image.fromarray(tensor)

## main

In [ ]:
# 데이터를 불러옵니다.
dr = DataReader("content.jpg", "style.jpg")

# Hub로부터 style transfer 모듈을 불러옵니다.
hub_module = hub.load(
    'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/1'
)

# 모듈에 이미지를 삽입해 Style Transfer를 실시합니다.
stylized_image = hub_module(dr.content, dr.style)[0]

# 결과를 출력합니다.
result = tensor_to_image(stylized_image)

# 결과를 저장합니다.
result.save("result.jpg")